### word2vec을 활용한 모델 구현

In [1]:
import pandas as pd

# 입력값은 단어로 표현된 리스트
DATA_IN_PATH = './data/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

# 전처리된 텍스트 데이터를 불러옴
train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)

reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

# 단어들의 리스트로 나눔
sentences = []
for review in reviews:
    sentences.append(review.split())

In [2]:
# word2vec을 학습하는 진행 상황을 확인해보기 위해 logging을 이용
import logging

# 학습 과정에서 로그 메시지를 양식에 맞게 INFO 수준으로 보여줌
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
# gensim 라이브러리의 word2vec을 활용해 모델 생성 및 학습을 실행
from gensim.models import word2vec

# 학습 시 필요한 하이퍼파라미터를 설정
num_features = 300     # 워드 벡터 특징값 수
min_word_count = 40    # 단어에 대한 최소 빈도 수
num_workers = 4        # 프로세스 개수
context = 10           # 컨텍스트 윈도 크기
downsampling = 1e-3    # 다움 샘플링 비율, 보통 0.001이 좋은 성능을 낸다고 알려져 있음

print('Training model ...')
model = word2vec.Word2Vec(sentences,
                         workers = num_workers,
                         size = num_features,
                         min_count = min_word_count,
                         window = context,
                         sample = downsampling)

2020-10-31 22:41:25,162 : INFO : collecting all words and their counts
2020-10-31 22:41:25,162 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


Training model ...


2020-10-31 22:41:25,518 : INFO : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2020-10-31 22:41:25,880 : INFO : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2020-10-31 22:41:26,058 : INFO : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2020-10-31 22:41:26,059 : INFO : Loading a fresh vocabulary
2020-10-31 22:41:26,113 : INFO : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2020-10-31 22:41:26,114 : INFO : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2020-10-31 22:41:26,146 : INFO : deleting the raw counts dictionary of 74065 items
2020-10-31 22:41:26,148 : INFO : sample=0.001 downsamples 30 most-common words
2020-10-31 22:41:26,149 : INFO : downsampling leaves estimated 2494384 word corpus (94.9% of prior 2627273)
2020-10-31 22:41:26,170 : INFO : estimated required memory for 8160 words and 300 dimen

In [4]:
# 모델의 하이퍼파라미터를 설정한 내용을 모델 이름에 담아 저장하면 나중에 참고하기 좋으며
# 모델을 저장하면 Word2Vec.load()를 통해 모델을 다시 사용할 수 있음
model_name = "300features_40minwords_10context"
model.save(model_name)

2020-10-31 22:41:38,720 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2020-10-31 22:41:38,722 : INFO : not storing attribute vectors_norm
2020-10-31 22:41:38,722 : INFO : not storing attribute cum_table
2020-10-31 22:41:38,904 : INFO : saved 300features_40minwords_10context


In [5]:
import numpy as np

def get_features(words, model, num_features):
    # words : 단어의 모음인 하나의 리뷰
    # model : 이미 학습한 word2cec 모델
    # num_features : word2vec으로 임베딩할 때 정했던 벡터의 차원 수
    
    # 출력 벡터 초기화
    feature_vector = np.zeros((num_features), dtype=np.float32)
    
    num_words = 0
    # 어휘사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words += 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vecotr = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [6]:
def get_dataset(reviews, model, num_features):
    # reviews : 학습 데이터인 전체 리뷰 데이터
    dataset = list()
    
    for s in reviews:
        dataset.append(get_features(s, model, num_features))
    
    reviewFeatureVecs = np.stack(dataset)
    
    return reviewFeatureVecs

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [8]:
# get_dataset()을 이용해 실제 학습에 사용될 입력값을 생성
test_data_vecs = get_dataset(sentences, model, num_features)

In [9]:
# 학습과 검증 데이터 분리
from sklearn.model_selection import train_test_split

RANDOM_SEED = 42
TEST_SPLIT = 0.2

X = test_data_vecs
y = np.array(sentiments)

X_train, X_eval, y_train, y_eval = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RANDOM_SEED)

In [10]:
# 모델 선언 및 학습
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

LogisticRegression(class_weight='balanced')

In [11]:
# 검증 데이터를 이용한 모델 성능 평가
print('Accuracy: {:f}'.format(lgs.score(X_eval, y_eval)))

Accuracy: 0.496200


In [12]:
# kaggle에 제출할 데이터 생성
TEST_CLEAN_DATA = 'test_clean.csv'
test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)

test_review = list(test_data['review'])

test_sentences = []
for review in test_review:
    test_sentences.append(review.split())

In [13]:
# 평가 데이터들을 word2vec으로 임베딩된 벡터값을 갖게 함
test_data_vecs = get_dataset(test_sentences, model, num_features)

In [14]:
# kaggle에 제출할 CSV파일을 만들어 저장
DATA_OUT_PATH = './submission/'

test_predicted = lgs.predict(test_data_vecs)
    
ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})
answer_dataset.to_csv(DATA_OUT_PATH + 'lgs_word2vec_submission.csv', index=False, quoting=3)